In [1]:
ipython = get_ipython()
colab   = 'google.colab' in str(ipython)
print(ipython)

if colab:
  from google.colab import drive
  drive.mount('/content/gdrive')
  %cd '/content/gdrive/My Drive/Colab Notebooks'

!ls "./"

def MODULE_FROM_NOTEBOOK(target, source, *keys): 
    from json import load
    with open(source) as notebook:
        data = load(notebook)
        with open(target,'w') as module:
          for cell in data['cells']:
              lines = cell['source']
              if type(lines) == str:
                  lines = lines.split('\n')
              line0  = (lines or [''])[0].strip()
              haskey = lambda key: line0 == f'### {key} ###'
              if cell['cell_type'] == 'code' and any(map(haskey,keys)):
                  code = '\n'.join(lines)
                  module.write(code)
                  module.write('\n')
                     
MODULE_FROM_NOTEBOOK('./ttt_tactics.py',
                     './ttt-tactics.ipynb',
                     'HEAD','GAME','DATA','MODEL','SEARCH')

from ttt_tactics import *

ipynb = 'ttt-tactics-train'

%load_ext tensorboard

!rm -rf "./tensorboard/"


Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks
'!!!.ipynb'    ttt-tactics	        ttt_tactics.py
 __pycache__   ttt-tactics.ipynb        ttt-tactics-test.ipynb
 ttt-3.ipynb   ttt-tactics-play.ipynb   ttt-tactics-train.ipynb
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9810827007565683556
,
 name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 15221889171543002605
physical_device_desc: "device: XLA_CPU device"
]
tensorflow: 2.3.0
DIR = ./ttt-tactics
MODULE =  ttt_tactics
<function savesamples at 0x7f240cebe400>
<function loadsamples at 0x7f240cebe620>
<class 'ttt_tactics.AlphaZeroModel'>
<function AlphaZeroPolicy at 0x7f240cebe840>


In [ ]:
### TRAIN ###

model   = AlphaZeroModel()
minloss = 2.3

models = sortedfiles(f'{DIR}/model-*.h5')
model.load(models[-1])
num = int(re.findall(r'\d+', str(model))[0]) 
print('num =', num)

path    = f'{DIR}/data-*.tfrec'
files   = sortedfiles(path)
dataset = loadsamples(files, buffer_size=200*1024, batch_size=1024, seed=None)
print('path =',path)
print('files:',len(files))

schedule = { 0:0.0001 }
def onschedule(epoch, lr): 
    if epoch in schedule: return schedule[epoch]  
    else:                 return lr
scheduler = LearningRateScheduler(onschedule)

# tensorboard = TensorBoard(log_dir        = f'./tensorboard/logs-{time()}', 
#                           histogram_freq = 1,
#                           write_graph    = True,
#                           write_images   = False,
#                           update_freq    = 'epoch', 
#                           profile_batch  = 0)

history = model.fit(
    x                   = dataset,
    epochs              = 10,
    verbose             = 1,
    callbacks           = [scheduler],
    workers             = 10,
    use_multiprocessing = False)
history = history.history

loss = history['loss'][-1]
ok   = loss < minloss
if ok:
    minloss = loss
    model.save(f'{DIR}/model-{(num+1):0>3d}.h5')

plt.figure()
plt.plot(history['loss'],        label='total loss')
plt.plot(history['policy_loss'], label='policy loss')
plt.plot(history['value_loss'],  label='value loss')
plt.title('training history')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(loc="upper left")
plt.savefig(f'{DIR}/{str(model)}.png')
plt.show()

ok = 'OK' if ok else 'NO'
NOTIFY(f'{str(model)} {ok}, loss: {loss}, shell={str(ipython)}', title=ipynb)


./ttt-tactics/model-008.h5 Oct 10 2020 13:18:11
num = 8
Instructions for updating:
Do not pass `graph_parents`.  They will  no longer be used.
Instructions for updating:
Do not call `graph_parents`.
path = ./ttt-tactics/data-*.tfrec
files: 46
Epoch 1/10
2204/2204 [==============================] - 2969s 1s/step - loss: 2.3111 - value_loss: 0.7411 - policy_loss: 1.5700 - value_lr: 1.0000e-04
Epoch 2/10
2204/2204 [==============================] - 2964s 1s/step - loss: 2.3111 - value_loss: 0.7411 - policy_loss: 1.5699 - value_lr: 1.0000e-04
Epoch 3/10
2204/2204 [==============================] - 2955s 1s/step - loss: 2.3106 - value_loss: 0.7408 - policy_loss: 1.5699 - value_lr: 1.0000e-04
Epoch 4/10
2204/2204 [==============================] - 2964s 1s/step - loss: 2.3105 - value_loss: 0.7407 - policy_loss: 1.5698 - value_lr: 1.0000e-04
Epoch 5/10
  91/2204 [>.............................] - ETA: 46:50 - loss: 2.3073 - value_loss: 0.7425 - policy_loss: 1.5649 - value_lr: 1.0000e-04

In [ ]:
model.save(f'{DIR}/model-{num:0>3d}.h5')